In [11]:
import pandas as pd
import pyarrow

from sqlalchemy import create_engine, text
from time import time

In [2]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet

--2024-08-08 18:40:08--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:263a:0:b:20a5:b140:21, 2600:9000:263a:6c00:b:20a5:b140:21, 2600:9000:263a:6e00:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:263a:0:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62553128 (60M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-05.parquet’

yellow_tripdata_202 100%[===================>]  59.66M  22.7MB/s    in 2.6s    

2024-08-08 18:40:11 (22.7 MB/s) - ‘yellow_tripdata_2024-05.parquet’ saved [62553128/62553128]



In [8]:
df = pd.read_parquet('yellow_tripdata_2024-05.parquet', engine='pyarrow')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-05-01 00:59:15,2024-05-01 01:23:50,1.0,6.10,1.0,N,138,145,1,28.20,7.75,0.5,5.00,0.00,1.0,42.45,0.0,1.75
1,2,2024-04-30 23:58:26,2024-05-01 00:29:42,1.0,11.23,1.0,N,138,249,1,46.40,6.00,0.5,8.72,0.00,1.0,66.87,2.5,1.75
2,2,2024-05-01 00:57:17,2024-05-01 01:14:15,1.0,9.02,1.0,N,138,170,1,35.90,6.00,0.5,10.57,6.94,1.0,65.16,2.5,1.75
3,2,2024-05-01 00:24:47,2024-05-01 00:48:51,1.0,6.53,1.0,N,87,133,1,30.30,1.00,0.5,7.06,0.00,1.0,42.36,2.5,0.00
4,2,2024-05-01 00:11:20,2024-05-01 00:52:10,1.0,14.38,1.0,N,161,165,1,61.80,1.00,0.5,0.00,0.00,1.0,66.80,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723828,2,2024-05-31 23:45:00,2024-06-01 00:09:00,NaN,4.07,NaN,None,88,186,0,29.93,0.00,0.5,0.00,0.00,1.0,33.93,NaN,NaN
3723829,2,2024-05-31 23:10:52,2024-05-31 23:38:05,NaN,4.81,NaN,None,231,143,0,30.71,0.00,0.5,0.00,0.00,1.0,34.71,NaN,NaN
3723830,2,2024-05-31 23:23:57,2024-05-31 23:38:35,NaN,2.03,NaN,None,142,140,0,17.15,0.00,0.5,4.23,0.00,1.0,25.38,NaN,NaN
3723831,2,2024-05-31 23:44:12,2024-05-31 23:47:44,NaN,0.94,NaN,None,141,263,0,-1.00,0.00,0.5,0.00,0.00,1.0,3.00,NaN,NaN


In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-05-01 00:59:15,2024-05-01 01:23:50,1.0,6.10,1.0,N,138,145,1,28.2,7.75,0.5,5.00,0.00,1.0,42.45,0.0,1.75
1,2,2024-04-30 23:58:26,2024-05-01 00:29:42,1.0,11.23,1.0,N,138,249,1,46.4,6.00,0.5,8.72,0.00,1.0,66.87,2.5,1.75
2,2,2024-05-01 00:57:17,2024-05-01 01:14:15,1.0,9.02,1.0,N,138,170,1,35.9,6.00,0.5,10.57,6.94,1.0,65.16,2.5,1.75
3,2,2024-05-01 00:24:47,2024-05-01 00:48:51,1.0,6.53,1.0,N,87,133,1,30.3,1.00,0.5,7.06,0.00,1.0,42.36,2.5,0.00
4,2,2024-05-01 00:11:20,2024-05-01 00:52:10,1.0,14.38,1.0,N,161,165,1,61.8,1.00,0.5,0.00,0.00,1.0,66.80,2.5,0.00


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723833 entries, 0 to 3723832
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

## connect to database

In [20]:
# Define the database connection parameters
db_user = 'root'
db_password = 'root'
db_host = 'localhost'  # or your database host
db_port = '5431'       # default PostgreSQL port
db_name = 'ny_taxi'

# Create the database connection string
connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Test the connection and check status
conn = engine.connect()
conn

## Create table in postgres with the data

In [15]:
print(pd.io.sql.get_schema(df, name='yellow_taxi_data'))

#df.print_schema() -> pyspark

CREATE TABLE "yellow_taxi_data" (
"VendorID" INTEGER,
  "tpep_pickup_datetime" TIMESTAMP,
  "tpep_dropoff_datetime" TIMESTAMP,
  "passenger_count" REAL,
  "trip_distance" REAL,
  "RatecodeID" REAL,
  "store_and_fwd_flag" TEXT,
  "PULocationID" INTEGER,
  "DOLocationID" INTEGER,
  "payment_type" INTEGER,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL,
  "congestion_surcharge" REAL,
  "Airport_fee" REAL
)


In [25]:
# result = conn.execute(text("SELECT table_name FROM information_schema.tables where table_schema='public'"))

# tables = result.fetchall()
# for table in tables:
#     print(table)

conn.execute(text("select * from yellow_taxi_data")).fetchall()

InternalError: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: select * from yellow_taxi_data]
(Background on this error at: https://sqlalche.me/e/20/2j85)